In [2]:
import numpy as np
import pandas as pd

In [3]:
np.set_printoptions(suppress=True, precision=2, linewidth=120)
np.warnings.filterwarnings('ignore')

In [4]:
def toMatrix(df):
    return df[df.columns[4:-1]].as_matrix()

In [5]:
def showClusterContent(kmeans, df, cluster_index):
    return df.iloc[np.where(kmeans.labels_ == cluster_index)[0], 1:].head(10)

In [6]:
def showCentroids(kmeans, df):
    # centroids = np.hstack((np.atleast_2d(range(cluster_n)).T, kmeans.cluster_centers_))
    pd.set_option('precision', 6)
    df = pd.DataFrame(kmeans.cluster_centers_, columns=spotify_df.columns[4:-1].tolist())
    df.index.names = ['Clusters']
    return df.T

In [9]:
from sklearn.metrics.pairwise import euclidean_distances
from IPython.core import display as ICD

def top_k_similar(kmeans, sp_df, records, cluster_n, top_k):
    all_dists = euclidean_distances(kmeans.cluster_centers_, records)
    for cluster in range(cluster_n):
        top_k_sim = np.argsort(all_dists[cluster])[::-1][:top_k]
        print ('Top Records for Cluster %d:' % cluster)
        ICD.display(sp_df.iloc[top_k_sim, [1,2,3,-1]])
        print ('='*100)
        print ('\n\n')

In [24]:
def show_word_clouds(kmeans, spotify_df, centroids_df, cluster_n):
    import urllib, cStringIO
    from PIL import Image
    import matplotlib.pyplot as plt

    from wordcloud import WordCloud, ImageColorGenerator

    features = spotify_df.columns[4:-1].tolist()
    centroids_df = centroids_df.T

    for cluster_index in range(cluster_n):
        words = {}
        for i in range(len(features)):
            words[features[i]] = centroids_df.iloc[cluster_index].tolist()[i]*5

        # counting the mood and ADDING MOOD to the WordCloud
        mood_n_artist_freq = spotify_df.iloc[np.where(kmeans.labels_ == cluster_index)[0], [3,-1]]
        # print(mood_n_artist_freq['mood(s)'])

        for mood in spotify_df['mood(s)'].unique().tolist():
            count = mood_n_artist_freq['mood(s)'].tolist().count(mood)
            words[mood] = count/10

        # ADDING Artist Names to the WordCloud -----------------------------
        for artist in spotify_df['artist'].unique().tolist():
            count = mood_n_artist_freq['artist'].tolist().count(artist)
            words[artist] = count
        # ------------------------------------------------------------------

        file = cStringIO.StringIO(urllib.urlopen("http://bmobasher.com/Class/cloud2.jpg").read())
        img = Image.open(file)
   
        alice_coloring = np.array(img)

        wc = WordCloud(background_color="white", max_words=200, mask=alice_coloring, max_font_size=55, random_state=42)
        # generate word cloud
        wc.generate_from_frequencies(words)

        # create coloring from image
        image_colors = ImageColorGenerator(alice_coloring)

        print ('Cluster %d:' % cluster_index)
        fig = plt.figure(1)
        fig.set_figwidth(15)
        fig.set_figheight(9)
        plt.imshow(wc, interpolation="bilinear")
        plt.axis('off')
        plt.show()
        fig.savefig("wordcloud_cluster_%d.png" % cluster_index, dpi=100)
        print('='*100)

In [2]:
def normalize_spotify(df):
    scaler = MinMaxScaler() 
    scaled_values = scaler.fit_transform(df[df.columns[4:-1]]) 
    return scaled_values